In [1]:
import numpy as np
import matplotlib.pyplot as plt
#
class Payoff:
    def payoff(self, x, xi, alpha, K, L, U, C, theta):
        # x = real space grid
        # xi = fourier space grid
        # alpha = dampening factor, + for call, - for put
        # L = lower trunc limit for option
        # U = upper trunc limit for option
        # C = stock price
        # theta = +1 call, -1 put

        # Scale
        S = C * np.exp(x)

        # Payoff - formula 1 above
        g = np.exp(alpha * x) * np.maximum(theta * (S - K), 0) * (S >= L) * (S <= U)

        # Analytical Fourier transform of the payoff
        l = np.log(L / C)  # lower log barrier
        k = np.log(K / C)  # log strike
        u = np.log(U / C)  # upper log barrier

        # Integration bounds
        if theta == 1:  # call
            a = max(l, k)
            b = u
        else:  # put
            a = min(k, u)
            b = l

        xi2 = alpha + 1j * xi
        # Formula 2
        G = C * ((np.exp(b * (1 + xi2)) - np.exp(a * (1 + xi2))) / (1 + xi2)
                - (np.exp(k + b * xi2) - np.exp(k + a * xi2)) / xi2)

        # Eliminable discontinuities for xi = 0, otherwise 0/0 = NaN
        if alpha == 0:
            G[len(x) // 2] = C * (np.exp(b) - np.exp(a) - np.exp(k) * (b - a))
        elif alpha == -1:
            G[len(x) // 2] = C * (b - a + np.exp(k - b) - np.exp(k - a))

        # Plot to compare the analytical and numerical payoffs
        gn = np.fft.fftshift(np.fft.fft(np.fft.ifftshift(G))) / ((x[1] - x[0]) * len(x))

        plt.plot(x, g, 'g', x, np.real(gn), 'r')
        plt.xlabel('x')
        plt.ylabel('g')
        if theta == 1:
            plt.title('Damped payoff function for a call option')
        else:  # put
            plt.title('Damped payoff function for a put option')
        plt.legend(['analytical', 'numerical'])
        plt.show()

        # return stock * exp(grid space), damped option payoff, FT of payoff
        return S, g, G